In [5]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20240130T133811 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


In [7]:
aws_role

'arn:aws:iam::058264089030:role/service-role/AmazonSageMaker-ExecutionRole-20240130T133811'

In [8]:
import boto3
import sagemaker as sm

sm_client = boto3.client("sagemaker")
sm_client.describe_notebook_instance(
NotebookInstanceName="cobalt-enefit"
)['InstanceType']

'ml.m5.12xlarge'

In [9]:
!pip install tsfresh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 33.3 MB/s eta 0:00:00


In [10]:
import pandas as pd

In [11]:
df = pd.read_csv(
    "data/consumption.csv", usecols=["prediction_unit_id", "datetime", "target"]
)[["prediction_unit_id", "datetime", "target"]].rename(
    columns={"prediction_unit_id": "unique_id", "datetime": "ds", "target": "y"}
)
df["ds"] = pd.to_datetime(df["ds"])
df.head()

,unique_id,ds,y
0,0,2021-09-01,96.590
1,1,2021-09-01,17.314
2,2,2021-09-01,656.859
3,3,2021-09-01,59.000
4,4,2021-09-01,501.760


In [12]:
df.isna().sum()

unique_id      0
ds             0
y            264
dtype: int64

In [13]:
df["y"] = df["y"].interpolate()

In [14]:
df.isna().sum()

unique_id    0
ds           0
y            0
dtype: int64

In [ ]:
from tsfresh import extract_features
extracted_features = extract_features(df, column_id="unique_id", column_sort="ds")

Feature Extraction:  88%|████████▊ | 61/69 [11:21<00:43,  5.42s/it]  

In [ ]:
extracted_features.head()